In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import torch

In [2]:
#separable 합성곱 함수
def separable_conv(x,inchannel,outchannel):
  x = keras.layers.Conv2D(inchannel,(3,3),strides=1,padding="same")(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Conv2D(outchannel,(1,1),strides=1,padding="same")(x)
  x = keras.layers.BatchNormalization()(x)
  return x

In [4]:
#model middle_flow 함수
def middle_flow(input_x):
  #encoder
  x = keras.layers.MaxPool2D((2,2),padding="same")(input_x)
  x = resiual_units(x)
  x = keras.layers.MaxPool2D((2,2),padding="same")(x)
  x = resiual_units(x)
  x = keras.layers.MaxPool2D((2,2),padding="same")(x)
  x = resiual_units(x)
  
  

  #decoder
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  
  x = separable_conv(x,x.shape[-1],512)
  x = separable_conv(x,x.shape[-1],512) 
  
  #sigmoid 
  x = keras.activations.sigmoid(x)
  x = keras.layers.Multiply()([input_x,x])
  x = keras.layers.Add()([input_x,x])

  x = resiual_units(x)

  return x

In [5]:
#resiual_units 함수 
def resiual_units(input_x):
  x = keras.layers.ReLU()(input_x)
  x = separable_conv(x,x.shape[-1],128)
  x = keras.layers.BatchNormalization()(x)

  x = keras.layers.ReLU()(x)
  x = separable_conv(x,x.shape[-1],256)
  x = keras.layers.BatchNormalization()(x)

  x = keras.layers.ReLU()(x)
  x = separable_conv(x,x.shape[-1],512)
  x = keras.layers.BatchNormalization()(x)
  
  input_x = keras.layers.Add()([x,input_x])

  return input_x

In [6]:
#골연령 측정 모델
#entry flow model
input = keras.Input(shape=(256,256,1))
x = keras.layers.Conv2D(32,(3,3),strides=2)(input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv2D(64,(3,3),strides=1)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
#첫번째
x1 = keras.layers.Conv2D(128,(1,1),strides=2)(x) 
x1 = keras.layers.BatchNormalization()(x1)

x = separable_conv(x,x.shape[-1],128)
x = keras.layers.ReLU()(x)

x = separable_conv(x,x.shape[-1],128)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])
#2번째
x1 = keras.layers.Conv2D(512,(1,1),strides=2)(x)
x1 = keras.layers.BatchNormalization()(x1)

x = separable_conv(x,x.shape[-1],512)
x = keras.layers.ReLU()(x)

x = separable_conv(x,x.shape[-1],512)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])


#middle flow model
x = middle_flow(x)


#exit flow model
x1 = keras.layers.Conv2D(1024,(1,1),strides=2)(x)

x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],728)
x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],1024)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])


x = separable_conv(x,x.shape[-1],1536)
x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],2048)
x = keras.layers.ReLU()(x)

x = keras.layers.GlobalAvgPool2D()(x)

x = keras.layers.Dense(1000,activation='relu')(x)
x = keras.layers.Dense(256,activation='relu')(x)
x = keras.layers.Dense(1)(x)

model = keras.models.Model(input,x)
model.compile(optimizer='adam',loss='mae',metrics=['mae','mse'])

In [ ]:
model.save('./tjnet_model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot
import graphviz

SVG(model_to_dot(model, show_shapes=True, show_layer_names=True, rankdir='TB',expand_nested=False, dpi=60, subgraph=False).create(prog='dot',format='svg'))

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [12]:
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

bones = np.load('/content/cboneage.npy',allow_pickle=True)
X_data = bones[:,2]
y = bones[:,1]


filename = 'checkpoint-50-epochs-16-batchs.h5'
checkpoint = ModelCheckpoint(filename,mointor='val_loss',verbose=1,save_best_only=True,mode='auto')


In [13]:
tmp = np.zeros((256,256))
for _,x in enumerate(X_data):
  if _ == 0:
    tmp = x
    tmp = tmp.reshape(1,256,256)
  else:
    tmp = np.concatenate([tmp,x.reshape(1,256,256)])
    
X_data = tmp

In [16]:
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.1f}".format(x)})
y = y.astype(np.float)

train_x,test_x,train_y,test_y = train_test_split(X_data.reshape(1205,256,256,-1),y,random_state=42,test_size=0.2)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,random_state=42,test_size=0.2)

In [20]:
with tf.device('/device:GPU:0'):
  model.fit(train_x,train_y,batch_size=16,epochs=150,callbacks=checkpoint,validation_data=(val_x,val_y))

Epoch 1/150
49/49 [==============================] - 45s 915ms/step - loss: 1.1653 - mae: 1.1653 - mse: 2.2619 - val_loss: 2.2775 - val_mae: 2.2775 - val_mse: 7.4966

Epoch 00001: val_loss did not improve from 1.35714
Epoch 2/150
49/49 [==============================] - 45s 909ms/step - loss: 0.8212 - mae: 0.8212 - mse: 1.2328 - val_loss: 2.7072 - val_mae: 2.7072 - val_mse: 9.7674

Epoch 00002: val_loss did not improve from 1.35714
Epoch 3/150
49/49 [==============================] - 45s 914ms/step - loss: 1.0455 - mae: 1.0455 - mse: 1.8425 - val_loss: 2.9642 - val_mae: 2.9642 - val_mse: 13.7671

Epoch 00003: val_loss did not improve from 1.35714
Epoch 4/150
49/49 [==============================] - 45s 914ms/step - loss: 1.0241 - mae: 1.0241 - mse: 1.8603 - val_loss: 3.9617 - val_mae: 3.9617 - val_mse: 20.4213

Epoch 00004: val_loss did not improve from 1.35714
Epoch 5/150
49/49 [==============================] - 45s 915ms/step - loss: 1.0221 - mae: 1.0221 - mse: 1.7776 - val_loss: 4.3

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 29/150
49/49 [==============================] - 45s 916ms/step - loss: 0.8993 - mae: 0.8993 - mse: 1.3908 - val_loss: 1.3333 - val_mae: 1.3333 - val_mse: 2.8045

Epoch 00029: val_loss did not improve from 1.23006
Epoch 30/150
49/49 [==============================] - 45s 912ms/step - loss: 0.9072 - mae: 0.9072 - mse: 1.3945 - val_loss: 1.3100 - val_mae: 1.3100 - val_mse: 3.0391

Epoch 00030: val_loss did not improve from 1.23006
Epoch 31/150
49/49 [==============================] - 45s 915ms/step - loss: 0.7496 - mae: 0.7496 - mse: 0.9506 - val_loss: 4.2365 - val_mae: 4.2365 - val_mse: 20.9079

Epoch 00031: val_loss did not improve from 1.23006
Epoch 32/150
49/49 [==============================] - 45s 914ms/step - loss: 0.9641 - mae: 0.9641 - mse: 1.6170 - val_loss: 1.2926 - val_mae: 1.2926 - val_mse: 2.6312

Epoch 00032: val_loss did not improve from 1.23006
Epoch 33/150
49/49 [==============================] - 45s 912ms/step - loss: 0.7047 - mae: 0.7047 - mse: 0.8337 - val_loss:

In [68]:
model.predict(bones[200,2].reshape(1,256,256,1))

array([[12.6]], dtype=float32)

In [64]:
train_y[200]

11.463013698630137

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)